In [1]:
import numpy as np
import os, sys
import h5py
import pandas as pd
import matplotlib.pyplot as plt

import datetime as DT
import matplotlib.dates as mdates


sys.path.remove(sys.path[0])
#print (sys.path)
sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/utils/')

#import readdata as rd
import plot_water_table2D
#sys.path.append(os.getcwd())

import importlib as imp
print (plot_water_table2D.__file__)

/Users/ajc/Core/codes/simulator/debug/ats/tools/utils/plot_water_table2D.py


In [2]:
import re
def sorted_nicely( l ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

In [3]:
path = "/Users/ajc/Core/simulations/barrow_icewedges/hydrologic_connection/ProjectionsK/"

In [4]:
Simulations = [['CESM_C1','CESM_C1A','CESM_C1B'],['CESM_C2'], ['CESM_C3', 'CESM_C3A', 'CESM_C3B', 'CESM_C3C']]
Simulations = [['CESM_C10','CESM_C11'], ['CESM_C20', 'CESM_C21','CESM_C22'],['CESM_C30', 'CESM_C31']]
Simulations = [['CESM_C10','CESM_C11'], ['CESM_C20', 'CESM_C21'],['CESM_C30', 'CESM_C31']]
Sim_name = ['S1','S2','S3']
Location = ['trough', 'center']

In [5]:
def getData(i,x_pos):
    time_, wl_, td_, z_ = dict(), dict(), dict(), dict()
    Time, WL,TD,z_elev = [], [],[],[]
    #location = Location[0]
    key = Sim_name[i] + '_' + str(x_pos)
    
    for s in Simulations[i]:
        
        infile = os.path.join(path,s)
        #print (infile)
        #if location_ == 'trough':
        #    t, wl, td, z = plot_water_table2D.water_table2D(infile, location='trough')
        #else:
        #t, wl, td, z = plot_water_table2D.water_table2D(infile)#,x_pos= 36) # center last cell
        t, wl, td, z = plot_water_table2D.water_table2D(infile,x_pos)  #trough
        
        a1 = len(t)%73
        a2 = len(t) - a1
        t = t[:a2]
        wl = wl[:a2]
        td = td[:a2]
        Time.append(t)
        WL.append(wl)
        TD.append(td)
        z_elev.append(z)
        
        #print ('TD:', i, len(td))
    #print ('-----------------------')
    #print ('Length1:',i, len(Time), len(WL), len(TD))
    #print ('FUll TD: ',i, len(td))
    if (x_pos == -1):
        key = Sim_name[i] + '_' + str(len(z_elev))
    time_[key+'_time'] = np.concatenate(Time)
    wl_[key+'_wl'] = np.concatenate(WL)
    td_[key+'_td'] = np.concatenate(TD)
    z_[key+'_z'] = np.array(z_elev)
    #print ('++++++++++++++++++++')
    print ('Length2:',i, x_pos,len(time_[key+'_time']), len(wl_[key+'_wl']), len(td_[key+'_td']))
    #print ('Length3:',i, time_[key+'_time'].shape, wl_[key+'_wl'].shape, td_[key+'_td'].shape)
    return time_,wl_, td_,z_

In [6]:
Time = dict()
WaterLevel = dict()
ThawDepth = dict()
Elevation = dict()

from multiprocessing import Pool
from functools import partial
num_processors = 3
#p=Pool(processes = num_processors)

#DataS1, DataS2, DataS3 = p.map(getData,[i for i in range(len(Simulations))])
#Data = [DataS1, DataS2,DataS3]

pool=Pool(processes = num_processors)

def call_data(x_pos):
    #print (loc_flag, pos)
    global DataS1, DataS2, DataS3
    prod_x=partial(getData, x_pos=x_pos)
    DataS1, DataS2, DataS3 = pool.map(prod_x,[i for i in range(len(Simulations))])

4 5038 45 85
4 5044 45 85
4 5261 45 85
4 1898 45 85
4 1679 45 85
Length2: 2 0 6935 6935 6935
Length2: 0 0 6935 6935 6935
4 1971 45 85
Length2: 1 0 7008 7008 7008
4 5044 45 85
4 5038 45 85
4 5261 45 85
4 1898 45 85
Length2: 2 0 6935 6935 6935
4 1679 45 85
Length2: 0 0 6935 6935 6935
4 1971 45 85
Length2: 1 0 7008 7008 7008


In [9]:

def getData_All(x_pos):
    #global DataSimOrig
    DataSimOrig = dict()
    Data = call_data(x_pos)
    #Data = [DataS1, DataS2,DataS3]
    #print (Data)
    for i in range(len(Sim_name)): #number of simulations
        #print ('HERE: ', i)
        for k in range(4): #number of keys = [time, water level, thaw depth, elevation]
            print (Data[i][k].keys())
            key = list(Data[i][k].keys())[0]
            z_k = list(Data[i][3].keys())[0]
            
            z = Data[i][3][z_k]
            
            if 'time' in key:
                DataSimOrig[key] = np.array([t for t in Data[i][k][key]])
                DataSimOrig[key+'_yr'] = np.reshape(DataSimOrig[key],(-1,73)).mean(axis=1)
                #print ('Time: ',DataSimOrig[key].shape)
            if 'wl' in key:
                DataSimOrig[key] = np.array([t + 0*z[0] for t in Data[i][k][key]])
                print ('WL:', key, DataSimOrig[key].shape,z, DataSimOrig[key])
                #print (DataSimOrig[key][1])
            if 'td' in key:
                DataSimOrig[key] = np.array([t + z[0] for t in Data[i][k][key]])
                
                DataSimOrig[key+'_max']  = np.reshape(DataSimOrig[key],(-1,73)).min(axis=1)
                #print ('TD:', DataSimOrig[key].shape)
                #print ('TD:', key, len(DataSimOrig[key]), len(Data[i][k][key]), DataSimOrig[key].shape)
            if 'z' in key:
                #print (key)
                DataSimOrig[z_k] = z
    return DataSimOrig

In [10]:
DataSimTrough = getData_All(x_pos=0)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
DataSimCenter = getData_All(-1)

In [ ]:
if (center):
    DataSimOrig = dict()
    for i in range(3):
        #print (Data[i][0].keys())
        for k in range(4):
            key = list(Data[i][k].keys())[0]
            z_k = list(Data[i][3].keys())[0]
            
            z = Data[i][3][z_k]
            
            if 'time' in key:
                DataSimOrig[key] = np.array([t for t in Data[i][k][key]])
                DataSimOrig[key+'_yr'] = np.reshape(DataSimOrig[key],(-1,73)).mean(axis=1)
                print ('Time: ',DataSimOrig[key].shape)
            if 'wl' in key:
                DataSimOrig[key] = np.array([t + z[0] for t in Data[i][k][key]])
                print ('WL:', DataSimOrig[key].shape,z)
                print (DataSimOrig[key][1])
            if 'td' in key:
                DataSimOrig[key] = np.array([t + z[0] for t in Data[i][k][key]])
                
                DataSimOrig[key+'_max']  = np.reshape(DataSimOrig[key],(-1,73)).min(axis=1)
                print ('TD:', DataSimOrig[key].shape)
                #print ('TD:', key, len(DataSimOrig[key]), len(Data[i][k][key]), DataSimOrig[key].shape)
            if 'z' in key:
                #print (key)
                DataSimOrig[z_k] = z
            #if k ==3:
            #    print (key,DataSimOrig['S1_center_td_max'].shape)  
    print (len(DataSimOrig['S1_center_td']),DataSimOrig['S1_center_td'].shape)

In [ ]:
if (trough):
    DataSimOrig = dict()
    for i in range(3):
        #print (Data[i][0].keys())
        for k in range(4):
            key = list(Data[i][k].keys())[0]
            z_k = list(Data[i][3].keys())[0]
            
            z = Data[i][3][z_k]
            
            if 'time' in key:
                DataSimOrig[key] = np.array([t for t in Data[i][k][key]])
                DataSimOrig[key+'_yr'] = np.reshape(DataSimOrig[key],(-1,73)).mean(axis=1)
                print ('Time: ',DataSimOrig[key].shape)
            if 'wl' in key:
                DataSimOrig[key] = np.array([t + z[0] for t in Data[i][k][key]])
                print ('WL:', DataSimOrig[key].shape,z)
                print (DataSimOrig[key][1])
            if 'td' in key:
                DataSimOrig[key] = np.array([t + z[0] for t in Data[i][k][key]])
                
                DataSimOrig[key+'_max']  = np.reshape(DataSimOrig[key],(-1,73)).min(axis=1)
                print ('TD:', DataSimOrig[key].shape)
                #print ('TD:', key, len(DataSimOrig[key]), len(Data[i][k][key]), DataSimOrig[key].shape)
            if 'z' in key:
                #print (key)
                DataSimOrig[z_k] = z
            #if k ==3:
            #    print (key,DataSimOrig['S1_center_td_max'].shape)  
#print (len(DataSimOrig['S1_center_td']),DataSimOrig['S1_center_td'].shape)

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/TroughBC/figures/'
def plot_data():
    fig, axs = plt.subplots(3,2, figsize=(12,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    axs = axs.ravel()
    #plt.axhline(y=Z_c,linestyle='--',color='g')
    DataSim = [DataSimCenter]#,DataSimTrough]

    year_on, thaw = True, False
    Colors = ['k','b','r','r']
    X = np.linspace(0,365,73)
    dry_year = 22
    wet_year =91
    avg_year = [27,44,76]
    Y = [wet_year,dry_year,avg_year[1]]
    avg_year = [27,44,76]
    Y = avg_year
    for k in range(1):
        #print ('K', k, DataSim[k])
        dat = DataSim[k]
        print (dat['S2_center_wl'])
        for j in range(3):
            s = 73*Y[j]
            e = 73*(Y[j]+1)

            for i in range(3):
                #t_k = list(Data[i][0].keys())[0] #+ '_yr'

                #wl_k = list(Data[i][1].keys())[0]
                #td_k = list(Data[i][2].keys())[0] + '_max'
                #z_k = list(Data[i][3].keys())[0]
                
                Keys = np.reshape(list(dat.keys()),(-1,6))
                #print (Keys)
                t_k, wl_k,z_k = Keys[i][0],Keys[i][2], Keys[i][5]
                z = dat[z_k] #Data[i][3][z_k]
                #print (t_k, wl_k, z_k)
                #print (len(dat[t_k][s:e]), len(dat[wl_k][s:e]))
                """
                if year_on and not thaw:
                    axs[j].plot(DataSimOrig[t_k][s:e],DataSimOrig[wl_k][s:e],label=Sim_name[i],color=Colors[i])
                elif not thaw:
                    plt.plot(DataSimOrig[t_k],DataSimOrig[wl_k],label=Sim_name[i])
                if (thaw):
                    plt.plot(DataSimOrig[t_k],DataSimOrig[td_k],label=Sim_name[i])
                """
                
                if year_on and not thaw:
                    print (2*(j)+k)
                    axs[2*j+k].plot(dat[t_k][s:e],dat[wl_k][s:e],label=Sim_name[i],color=Colors[i])
                #elif not thaw:
                #    plt.plot(DataSimOrig[t_k],DataSimOrig[wl_k],label=Sim_name[i])
                #if (thaw):
                #    plt.plot(DataSimOrig[t_k],DataSimOrig[td_k],label=Sim_name[i])
                    
                #axs[2*j+k].axhline(y=z[0],linestyle='--',color='g')
                #axs[2*j+k].set_ylim(4.4,5.0)
                #axs[2*j+k].set_xlim((Y[j], Y[j]+1))
                #axs[2*j+k].set_xticklabels(np.arange(0,366,73))
        #plt.legend()
        #plt.savefig(outfile+'watertable.pdf', bbox_inches='tight', dpi=400)
plt.show()

#plot 2nd row for high snow years

In [ ]:
plot_data()

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/TroughBC/figures/'

fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
#axs = axs.ravel()
#plt.axhline(y=Z_c,linestyle='--',color='g')
Z_c = DataSimOrig[z_k][0,0]
year_on, thaw = True, True
Colors = ['k','b','r','r']
X = np.linspace(0,365,73)
dry_year = 22
wet_year =91
avg_year = [27,44,76]
Y = [wet_year,dry_year,avg_year[1]]
avg_year = [27,44,76]
Y = avg_year
def get_moving_average(key, m=5):
    d_avg = []
    len1 = len(DataSimOrig[key])
    for i in range(len1):
        if i < len1 - 5:
            s1 = np.mean(DataSimOrig[td_k][i:i+5])
        else:
            s1 = np.mean(DataSimOrig[td_k][i:i+5])
        d_avg.append(s1)
    return d_avg
for j in range(3):
    t_k = list(Data[i][0].keys())[0] + '_yr'
    td_k = list(Data[j][2].keys())[0] + '_max'
    dd = get_moving_average(td_k)
    #plt.plot(DataSimOrig[t_k],DataSimOrig[td_k],color=Colors[j],label=Sim_name[j])
    ###plt.plot(2006 + DataSimOrig[t_k],dd,color=Colors[j],label=Sim_name[j])
    #plt.axhline(y=Z_c,linestyle='--',color='g')
    #print (Z_c)
    xx = (dd[0] - dd[-1])/dd[0]*100
    ddd = [k - dd[0] for k in dd]
    print (xx, dd[0]-dd[-1])
    plt.plot(2006 + DataSimOrig[t_k],dd - Z_c,color=Colors[j],label=Sim_name[j])
    axs.set_ylim(-2.25,0)
    #plt.gca().invert_yaxis()
    axs.set_yticks(np.arange(0,-2.25,-0.5))
    xlabels = np.arange(0,2.25,0.5)
    #print (xlabels)
    axs.set_yticklabels(xlabels)
    #plt.ylim(3.2,3.5)
    plt.xlabel('Calendar year',fontsize=12, fontweight='normal')
    plt.ylabel('Maximum thaw depth [m]',fontsize=12, fontweight='normal')
    #plt.xlim((2006, 2100))
    #axs[j].set_xlim((Y[j], Y[j]+1))
    #axs[j].set_xticklabels(np.arange(0,366,73))
plt.legend()
#plt.savefig(outfile+'thawdepth-trough.pdf', bbox_inches='tight', dpi=800)
plt.show()

#plot 2nd row for high snow years

In [ ]:
x= (1.68 - 1.44)/1.44*100
x

In [ ]:
#how trough water table can impact polygon turn over
# consider different peat in the center, trough and rim. since we see some differences in the trough thaw depth so
# possible that polygon turnover will be affected
# make a polygon turn over phenomenological model related to thaw depth at the center, trough and rim, and
# wet peat and more peat in the trough can lead to deeper ALT,

In [ ]:
#Effect of wetter/drier ice wedge polygon center on ice wedge and polygon trough
# Even fine-scale simulations need large scale flow behavior
# ----- PLOTS -----
# water table in the center and in the trough
# evaporation [from the center, from the trough, entire polygon]
# water content in the ALT
# does it affect thaw timing, delayed fall freeze-up
# pick a year and plot water table in the center for all cases, see responses of center water table to rain
# pick a year and plot flux through vertial_rim, how flow directions is altered by run-on, run-off in rain event
# discharge through trough boundary
# Thaw depth [not affected?]